In [63]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time

# initialize sheet with gspread
work_sheet = "UT Fall 2019 Scorecard"
scope = ["https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)
client = gspread.authorize(credentials)
sheet = client.open(work_sheet).get_worksheet(1)

# urls of interest
POST_LOGIN_URL = 'https://venmo.com/groups/sign-in'
REQUEST_URL = 'https://venmo.com/TXOmegas'

# chromedriver initialization
chromedriver_path = "D:\Brian\Documents\chromedriver"
driver = webdriver.Chrome(chromedriver_path)

# initial login post and credentials
driver.get(POST_LOGIN_URL)
username = driver.find_element_by_name("username")
password = driver.find_element_by_name("password")
username.send_keys("") # username here
password.send_keys("") # password here
driver.find_element_by_css_selector(".button").click()

# 60 seconds for 2FA, signal on homepage arrival
home_page = "https://venmo.com/"
success = False
try:
    element = WebDriverWait(driver, 60).until(EC.url_to_be(home_page))
    print("homepage arrived")
    driver.get(REQUEST_URL)
    if driver.current_url == REQUEST_URL:
        success = True
except:
    print("homepage error")
    driver.quit()


homepage arrived


In [64]:
def print_trans_list(trans_list):
    for j in range (len(trans_list)):
        print(str(j) + ": " + trans_list[j].text)

In [65]:
if success:
    feed_xpath = "/html/body/div[2]/div[8]/div[1]/div[4]/div[2]"
    trans_list = []
    # allow elements to load
    time.sleep(2)
    for i in range (2, 17):
        trans_xpath = "/div[" + str(i) + "]"
        trans_list.append(driver.find_element_by_xpath(feed_xpath + trans_xpath))
print_trans_list(trans_list)
    

0: UT Omega Phi Gamma charged Chandler Yoon
for (1) Formal Shirt
Like · December 19 ·
+$18.00
1: Brian LeQuang paid UT Omega Phi Gamma
Transfer
Like · December 17 ·
+$9.07
2: UT Omega Phi Gamma charged Brian Nguyen
for (2) Formal Shirts
Like · December 16 ·
+$36.00
3: UT Omega Phi Gamma charged Brian Nguyen
for (2) DD Shirts
Like · December 16 ·
+$30.00
4: UT Omega Phi Gamma charged Stephen Du
for (2) Formal Shirts
Like · December 14 ·
+$36.00
5: Jeremy Tajonera paid UT Omega Phi Gamma
o 👕
Like · December 11 ·
+$17.00
6: Matthew Koh charged UT Omega Phi Gamma
Lock-in
Like · December 10 ·
♥ Brian Nguyen likes this.
-$98.00
7: Eric Dulin charged UT Omega Phi Gamma
25*11 new guys
Like · December 10 ·
-$275.00
8: Andrew Vo paid UT Omega Phi Gamma
🤶🏼🌲
Like · December 9 ·
Andrew Vo: Dddddooooos
December 9
+$150.00
9: UT Omega Phi Gamma charged John Bao
for (2) Formal Shirts
Like · December 7 ·
+$36.00
10: UT Omega Phi Gamma charged Brandon Nailling
for (1) Formal Shirt
Like · December 6 ·
+$

In [66]:
class Transaction:
    def __init__ (this, date, name, desc, dollar):
        this.date = date
        this.name = name
        this.desc = desc
        this.dollar = dollar
    
    def show_trans(this):
        print("date:   " + this.date + "\n" +
              "name:   " + this.name + "\n" +
              "desc:   " + this.desc + "\n" +
              "dollar: " + this.dollar + "\n")

In [69]:
account_name = "UT Omega Phi Gamma"
formatted_trans = []
# parse each transaction element from trans_list
for elem in reversed (trans_list):
    # parse date and reformat
    date = elem.get_attribute("rel")
    date = date[5:10]
    date = date.replace("-", "/")
    
    tokenize = elem.text.split("\n")
    
    # parse for person's name
    token = " charged "
    if "paid" in tokenize[0]:
        token = " paid "
    personel = tokenize[0].split(token, 1)
    name = personel[0]
    if name == account_name:
        name = personel[1]
        
    # parse description
    desc = tokenize[1]
    
    # parse for dollar amount
    dollar = tokenize[-1]
    dollar_int = ""
    if "+" in tokenize[3]:
        dollar_int = dollar[2:len(dollar)]
    else:
        dollar_int = dollar.replace("$", "")
    
    formatted_trans.append(Transaction(date, name, desc, dollar_int))
    
for i in range(len(formatted_trans)):
    print("Transaction #" + str(i) + ":")
    formatted_trans[i].show_trans()


Transaction #0:
date:   12/06
name:   Chris Chung
desc:   Sp19 due pt 3
dollar: 150.00

Transaction #1:
date:   12/06
name:   Andrew Kim
desc:   A&M present gifts
dollar: -58.84

Transaction #2:
date:   12/07
name:   Matthew Koh
desc:   for the plebs
dollar: -440.00

Transaction #3:
date:   12/07
name:   Josh Tsai
desc:   for (1) Formal Shirt
dollar: 18.00

Transaction #4:
date:   12/07
name:   Brandon Nailling
desc:   for (1) Formal Shirt
dollar: 18.00

Transaction #5:
date:   12/07
name:   John Bao
desc:   for (2) Formal Shirts
dollar: 36.00

Transaction #6:
date:   12/10
name:   Andrew Vo
desc:   🤶🏼🌲
dollar: +150.00

Transaction #7:
date:   12/10
name:   Eric Dulin
desc:   25*11 new guys
dollar: -275.00

Transaction #8:
date:   12/10
name:   Matthew Koh
desc:   Lock-in
dollar: -98.00

Transaction #9:
date:   12/12
name:   Jeremy Tajonera
desc:   o 👕
dollar: 17.00

Transaction #10:
date:   12/15
name:   Stephen Du
desc:   for (2) Formal Shirts
dollar: 36.00

Transaction #11:
date:   